# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import re

# Files Loading

We load all the files in the folder "data"

In [2]:
folder = 'data'
files = os.listdir(folder)

dfs = []

for file in files:
    
    path = os.path.join(folder, file)
    df = pd.read_excel(path, header=0)
    dfs.append(df)


Laptops = pd.concat(dfs)

# Data Cleaning

First of all, we need to delete all the duplicated values we could have. The best column to do so is the Laptop column:

In [3]:
Laptops = Laptops.drop_duplicates(subset='Laptop')

In [4]:
Laptops

,Laptop,New Price,Old Price,Price,Old Price 2,Status
0,ASUS ExpertBook B1 B1502CBA-EJ0436X Intel Core...,699,1.009,NaN,PVPR,Patrocinado
1,Alurin Go Start Intel Celeron N4020/8GB/256GB ...,229,299,NaN,PVPR,Envío gratis
2,ASUS ExpertBook B1 B1502CBA-EJ0424X Intel Core...,549,789,NaN,PVPR,Patrocinado
3,MSI Katana GF66 12UC-082XES Intel Core i7-1270...,899,1.199,NaN,PVPR,Envío gratis
4,HP 15S-FQ5085NS Intel Core i5-1235U/16GB/512GB...,NaN,NaN,"669,01€",NaN,Patrocinado
...,...,...,...,...,...,...
2159,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,"2.151,39€","2.699,99€",NaN,PVPR,Reacondicionado
2160,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,"2.016,39€","2.899,99€",NaN,PVPR,Reacondicionado
2161,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,"2.069,69€","3.399,99€",NaN,PVPR,Reacondicionado
2162,Razer Book 13 Intel Evo Core i7-1165G7/16GB/1T...,"1.515,19€","1.899,99€",NaN,PVPR,Reacondicionado


In a first sight, we see the values in Pricing columns are awful. Let's concentrate in the first one, New Price:

In [5]:
Laptops['New Price'].head(30)

0         699
1         229
2         549
3         899
4         NaN
5       1.349
6         NaN
7         529
8         329
10        879
11        320
12        999
13        829
14        NaN
15        319
16        549
17        709
18    458,78€
19        469
20        799
21        999
22        599
23        NaN
24        739
25        339
26        849
27        219
28        NaN
29        599
30        NaN
Name: New Price, dtype: object

The only problem I see in New Price is the € symbol. Values such '1.099,99€' are a challenge, since includes ".", "," and "€".

In [6]:
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if '€' in x:
        return float(x.replace('€', '').replace('.','').replace(',','.'))
    return 0.0

df['New Price'] = Laptops['New Price'].apply(value_to_float)
df['New Price'] = df['New Price'].apply(lambda x: x * 1000 if x < 10 else x)

In [7]:
Laptops['New Price'] = df['New Price']

Now let's dive into the next column, Old Price:

In [8]:
Laptops['Old Price'].head(20)

0     1.009
1       299
2       789
3     1.199
4       NaN
5     1.699
6       NaN
7       NaN
8       519
10    1.149
11      349
12    1.399
13    1.199
14      NaN
15      NaN
16      699
17      799
18      549
19      NaN
20      999
Name: Old Price, dtype: object

It looks very similar to New Price. The function used before (value_to_float(x)) should work:

In [9]:
df['Old Price'] = Laptops['Old Price'].apply(value_to_float)
df['Old Price'] = df['Old Price'].apply(lambda x: x * 1000 if x < 10 else x)

In [10]:
df['Old Price'].head(20)

0     1009.0
1      299.0
2      789.0
3     1199.0
4        NaN
5     1699.0
6        NaN
7        NaN
8      519.0
9        NaN
10    1149.0
11     349.0
12    1399.0
13    1199.0
14       NaN
15       NaN
16     699.0
17     799.0
18     549.0
19       NaN
Name: Old Price, dtype: float64

Looks nice, so we move it to Laptops df:

In [11]:
Laptops['Old Price'] = df['Old Price']

The next column is Price. Let's see how it looks like:

In [12]:
Laptops['Price'].head(20)

0                       NaN
1                       NaN
2                       NaN
3                       NaN
4                   669,01€
5                       NaN
6                       909
7     Más bajo 30 días 549€
8                       NaN
10                      NaN
11                      NaN
12                      NaN
13                      NaN
14                  476,99€
15    Más bajo 30 días 279€
16                      NaN
17                      NaN
18                      NaN
19    Más bajo 30 días 499€
20                      NaN
Name: Price, dtype: object

Looks similar to Old Price, but we also have values with the string "Más bajo 30 días ". We define a function to clean it, including this, as we did in New Price:

In [13]:
def value_to_float_2(x):
    if type(x) == float or type(x) == int:
        return x
    if 'bajo' in x:
        return float(x.replace('Más bajo 30 días ', '').replace('€', '').replace('.','').replace(',','.'))
    if '€' in x:
        return float(x.replace('€', '').replace('.','').replace(',','.'))
    return 0.0

df['Price'] = Laptops['Price'].apply(value_to_float_2)
df['Price'] = df['Price'].apply(lambda x: x * 1000 if x < 10 else x)

In [14]:
df['Price'].head(20)

0        NaN
1        NaN
2        NaN
3        NaN
4     669.01
5        NaN
6     909.00
7     549.00
8        NaN
9        NaN
10       NaN
11       NaN
12       NaN
13       NaN
14    476.99
15    279.00
16       NaN
17       NaN
18       NaN
19    499.00
Name: Price, dtype: float64

Looks perfect, we move it to Laptops df:

In [15]:
Laptops['Price'] = df['Price']

Let's see the last one, Old Price 2:

In [16]:
Laptops['Old Price 2'].head(20)

0        PVPR
1        PVPR
2        PVPR
3        PVPR
4         NaN
5        PVPR
6         NaN
7     809,01€
8        PVPR
10       PVPR
11       PVPR
12       PVPR
13       PVPR
14        NaN
15        391
16       PVPR
17       PVPR
18       PVPR
19        769
20       PVPR
Name: Old Price 2, dtype: object

It looks like the other ones, but additionally we need to replace the string "PVPR". Let's build a function:

In [17]:
def value_to_float_3(x):
    if type(x) == float or type(x) == int:
        return x
    if 'PVPR' in x:
        return float(x.replace('PVPR', '0'))
    if '€' in x:
        return float(x.replace('€', '').replace('.','').replace(',','.'))
    return float(0.0)

df['Old Price 2'] = Laptops['Old Price 2'].apply(value_to_float_3)
df['Old Price 2'] = df['Old Price 2'].apply(lambda x: x * 1000 if x < 10 else x)

In [18]:
df['Old Price 2'].head(20)

0       0.00
1       0.00
2       0.00
3       0.00
4        NaN
5       0.00
6        NaN
7     809.01
8       0.00
9        NaN
10      0.00
11      0.00
12      0.00
13      0.00
14       NaN
15    391.00
16      0.00
17      0.00
18      0.00
19    769.00
Name: Old Price 2, dtype: float64

Now we have nan values and 0.0 values. It's not important for our purpose, so let's go ahead.

In [19]:
Laptops['Old Price 2'] = df['Old Price 2']

In [20]:
Laptops

,Laptop,New Price,Old Price,Price,Old Price 2,Status
0,ASUS ExpertBook B1 B1502CBA-EJ0436X Intel Core...,699.00,1009.00,NaN,0.0,Patrocinado
1,Alurin Go Start Intel Celeron N4020/8GB/256GB ...,229.00,299.00,NaN,0.0,Envío gratis
2,ASUS ExpertBook B1 B1502CBA-EJ0424X Intel Core...,549.00,789.00,NaN,0.0,Patrocinado
3,MSI Katana GF66 12UC-082XES Intel Core i7-1270...,899.00,1199.00,NaN,0.0,Envío gratis
4,HP 15S-FQ5085NS Intel Core i5-1235U/16GB/512GB...,NaN,NaN,669.01,NaN,Patrocinado
...,...,...,...,...,...,...
2159,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,2151.39,2699.99,NaN,0.0,Reacondicionado
2160,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,2016.39,2899.99,NaN,0.0,Reacondicionado
2161,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,2069.69,3399.99,NaN,0.0,Reacondicionado
2162,Razer Book 13 Intel Evo Core i7-1165G7/16GB/1T...,1515.19,1899.99,NaN,0.0,Reacondicionado


# Working with prices 

Now we should transfer the prices. The raw structure is as follows:

- #### New Price: Discounted price. Not interesting, since it's not the common price.
- #### Old Price: Common price, in the cases a New Price exists.
- #### Price: Price when New Price nor Old Price exists.
- #### Old Price 2: Common price in the cases there is a Price and New Price.

The goal now is to create a column 'Final Price', showing the common price, without any kind of discount.

So the 'Final Price' should be the highest price from the four pricing columns.

In [21]:
Laptops['Final Price'] = Laptops[['Old Price', 'Old Price 2', 'New Price', 'Price']].max(axis=1)

In [22]:
Laptops

,Laptop,New Price,Old Price,Price,Old Price 2,Status,Final Price
0,ASUS ExpertBook B1 B1502CBA-EJ0436X Intel Core...,699.00,1009.00,NaN,0.0,Patrocinado,1009.00
1,Alurin Go Start Intel Celeron N4020/8GB/256GB ...,229.00,299.00,NaN,0.0,Envío gratis,299.00
2,ASUS ExpertBook B1 B1502CBA-EJ0424X Intel Core...,549.00,789.00,NaN,0.0,Patrocinado,789.00
3,MSI Katana GF66 12UC-082XES Intel Core i7-1270...,899.00,1199.00,NaN,0.0,Envío gratis,1199.00
4,HP 15S-FQ5085NS Intel Core i5-1235U/16GB/512GB...,NaN,NaN,669.01,NaN,Patrocinado,669.01
...,...,...,...,...,...,...,...
2159,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,2151.39,2699.99,NaN,0.0,Reacondicionado,2699.99
2160,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,2016.39,2899.99,NaN,0.0,Reacondicionado,2899.99
2161,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,2069.69,3399.99,NaN,0.0,Reacondicionado,3399.99
2162,Razer Book 13 Intel Evo Core i7-1165G7/16GB/1T...,1515.19,1899.99,NaN,0.0,Reacondicionado,1899.99


In [23]:
Laptops['Final Price'].head(20)

0     1009.00
1      299.00
2      789.00
3     1199.00
4      669.01
5     1699.00
6      909.00
7      809.01
8      519.00
10    1149.00
11     349.00
12    1399.00
13    1199.00
14     476.99
15     391.00
16     699.00
17     799.00
18     549.00
19     769.00
20     999.00
Name: Final Price, dtype: float64

The new column looks perfect

# Defining Laptops caractheristics

Now, we should define some Laptop caracteristics. Important columns would be:

- #### Brand: Apple, MSI, Asus, etc
- #### Model: Macbook, Katana, ExpertBook, etc
- #### CPU: Intel Core i7, AMD Ryzen 5, etc
- #### RAM: Integer value (4, 8, 16, etc)
- #### Storage: Integer value (512, 256, 1024, etc)
- #### Storage type: HDD or SDD
- #### Screen: float value (15.6, 17.3, 16.1, etc)
- #### GPU: RTX 3060, RTX 3050 Ti, RTX 4070, etc
- #### Touch: Yes, No

So let's work on it

In [24]:
Laptops['Laptop'].head(20)

0     ASUS ExpertBook B1 B1502CBA-EJ0436X Intel Core...
1     Alurin Go Start Intel Celeron N4020/8GB/256GB ...
2     ASUS ExpertBook B1 B1502CBA-EJ0424X Intel Core...
3     MSI Katana GF66 12UC-082XES Intel Core i7-1270...
4     HP 15S-FQ5085NS Intel Core i5-1235U/16GB/512GB...
5     MSI Crosshair 17 C12VF-264XES Intel Core i7-12...
6     Lenovo Thinkpad E14 Gen 4 AMD Ryzen 5 5625U/8G...
7     ASUS VivoBook 15 F515JA-EJ2882W Intel Core i7-...
8     Medion Akoya E15415 Intel Core i5-10210U/8GB/2...
10    HP Victus 16-d1038ns Intel Core i7-12700H/16GB...
11    Lenovo V15 IGL Intel Celeron N4020/8GB/256 GB ...
12    MSI Thin GF63 12VE-021XES Intel Core i7-12650H...
13    ASUS ROG Strix G15 G513RC-HN088 AMD Ryzen 7 68...
14    Lenovo V15 G3 ABA AMD Ryzen 5 5625U/16GB/512GB...
15    Lenovo IdeaPad 1 15ADA7 AMD Ryzen 3 3250U/8GB/...
16    HP 15S-FQ5013NS Intel Core i5-1235U/8GB/512GB ...
17    Lenovo IdeaPad 3 15IAU7 Intel Core i5-1235U/16...
18    HP 15S-FQ2163NS Intel Core i3-1115G4/8GB/5

## Brand

In this column, we've created the dictionary file master_data\brands.xlsx. Let's see how it looks like:

In [25]:
path_brands = os.path.join('master_data', 'brands.xlsx')

brands_df = pd.read_excel(path_brands)
brands_df

,Keyword,Brand
0,Asus,Asus
1,MSI,MSI
2,Lenovo,Lenovo
3,Apple,Apple
4,Dell,Dell
5,Acer,Acer
6,Medion,Medion
7,Razer,Razer
8,LG,LG
9,Samsung,Samsung


Perfect. Let's generate the new column Brand, searching the values Brand in Laptop. The coincidence will be set as Brand:

In [26]:
brand_dict = brands_df.set_index('Keyword')['Brand'].to_dict()

def assign_brand(row):
    for keyword, brand in brand_dict.items():
        if keyword.lower() in row['Laptop'].lower():
            return brand
    return None

Laptops['Brand'] = Laptops.apply(assign_brand, axis=1)

In [27]:
Laptops['Brand'].value_counts()

Asus                415
HP                  368
Lenovo              366
MSI                 308
Acer                137
Apple               116
Dell                 84
Microsoft            77
Gigabyte             48
Razer                37
Medion               32
LG                   32
Alurin               29
PcCom                24
Samsung              22
Dynabook Toshiba     19
Vant                 11
Primux                8
Deep Gaming           8
Innjoo                6
Thomson               4
Prixton               3
Millenium             2
Denver                1
Jetwing               1
Realme                1
Toshiba               1
Name: Brand, dtype: int64

In [28]:
null_values = Laptops[Laptops['Brand'].isna()]
null_values['Laptop'].tolist()

[]

Looks nice!

## CPU 

Now, in order to simplify it, let's generate the column 'CPU'. This column will help us further to generate the column 'Model'. In order to generate CPU, we will use the dictionary cpu.xlsx

In [29]:
path_cpu = os.path.join('master_data', 'cpu.xlsx')

cpu_df = pd.read_excel(path_cpu)
cpu_df

,Keyword,CPU
0,Intel Core i7,Intel Core i7
1,Intel Core i7,Intel Core i7
2,Intel Evo Core i7,Intel Evo Core i7
3,Intel Evo Core i7,Intel Evo Core i7
4,Intel Core i5,Intel Core i5
5,Intel Core i5,Intel Core i5
6,Ryzen 5,AMD Ryzen 5
7,Ryzen 7,AMD Ryzen 7
8,Intel Core i9,Intel Core i9
9,Intel Core i3,Intel Core i3


In [30]:
cpu_dict = cpu_df.set_index('Keyword')['CPU'].to_dict()

def assign_cpu(row):
    for keyword, cpu in cpu_dict.items():
        if keyword.lower() in row['Laptop'].lower():
            return cpu
    return None

Laptops['CPU'] = Laptops.apply(assign_cpu, axis=1)

In [31]:
Laptops['CPU'].value_counts()

Intel Core i7            710
Intel Core i5            535
AMD Ryzen 7              156
Intel Core i3            130
AMD Ryzen 5              127
Intel Core i9             94
Intel Celeron             94
Intel Evo Core i7         82
AMD Ryzen 9               44
AMD Ryzen 3               44
Intel Evo Core i5         30
Apple M2                  28
AMD 3020e                 13
Apple M2 Pro              13
Apple M1                  11
AMD Athlon                10
Intel Pentium             10
Apple M1 Pro               7
Intel Core M3              5
AMD 3015e                  3
Microsoft SQ1              3
Qualcomm Snapdragon 7      3
AMD Radeon 9               2
Qualcomm Snapdragon 8      2
Intel Evo Core i9          1
Mediatek MT8183            1
AMD 3015Ce                 1
AMD Radeon 5               1
Name: CPU, dtype: int64

In [32]:
null_values = Laptops[Laptops['CPU'].isna()]
null_values['Laptop'].tolist()

[]

Looks perfect!

## Model

The idea to get the Model, is the same as the previous caractheristics. We search for the models in dictionary model.xlsx

In [33]:
path_model = os.path.join('master_data', 'model.xlsx')

model_df = pd.read_excel(path_model)
model_df

,Keyword,Model
0,MacBook Pro,MacBook Pro
1,MacBook Air,MacBook Air
2,250,250
3,V15,V15
4,Flex Advance,Flex Advance
...,...,...
131,V330,V330
132,Alpha,Alpha
133,Delta,Delta
134,GL65,GL65


In [34]:
model_dict = model_df.set_index('Keyword')['Model'].to_dict()

def assign_model(row):
    for keyword, model in model_dict.items():
        if keyword.lower() in row['Laptop'].lower():
            return model
    return None

Laptops['Model'] = Laptops.apply(assign_model, axis=1)

In [35]:
Laptops['Model'].value_counts()

15S            115
IdeaPad        104
ROG            101
ThinkPad        99
VivoBook        99
              ... 
100w             1
Electronics      1
Cyborg           1
14S              1
GL75             1
Name: Model, Length: 121, dtype: int64

In [36]:
null_values = Laptops[Laptops['Model'].isna()]
null_values['Laptop'].tolist()

[]

In [37]:
Laptops

,Laptop,New Price,Old Price,Price,Old Price 2,Status,Final Price,Brand,CPU,Model
0,ASUS ExpertBook B1 B1502CBA-EJ0436X Intel Core...,699.00,1009.00,NaN,0.0,Patrocinado,1009.00,Asus,Intel Core i5,ExpertBook
1,Alurin Go Start Intel Celeron N4020/8GB/256GB ...,229.00,299.00,NaN,0.0,Envío gratis,299.00,Alurin,Intel Celeron,Go
2,ASUS ExpertBook B1 B1502CBA-EJ0424X Intel Core...,549.00,789.00,NaN,0.0,Patrocinado,789.00,Asus,Intel Core i3,ExpertBook
3,MSI Katana GF66 12UC-082XES Intel Core i7-1270...,899.00,1199.00,NaN,0.0,Envío gratis,1199.00,MSI,Intel Core i7,Katana
4,HP 15S-FQ5085NS Intel Core i5-1235U/16GB/512GB...,NaN,NaN,669.01,NaN,Patrocinado,669.01,HP,Intel Core i5,15S
...,...,...,...,...,...,...,...,...,...,...
2159,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,2151.39,2699.99,NaN,0.0,Reacondicionado,2699.99,Razer,Intel Core i7,Blade
2160,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,2016.39,2899.99,NaN,0.0,Reacondicionado,2899.99,Razer,Intel Core i7,Blade
2161,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,2069.69,3399.99,NaN,0.0,Reacondicionado,3399.99,Razer,Intel Core i7,Blade
2162,Razer Book 13 Intel Evo Core i7-1165G7/16GB/1T...,1515.19,1899.99,NaN,0.0,Reacondicionado,1899.99,Razer,Intel Evo Core i7,Book


Looks good!

## RAM

The strategy for this column is to extract the digits just before "GB/" and "Gb/".

In [38]:
Laptops['RAM'] = Laptops['Laptop'].str.extract(r'(\d+)\s*(?:GB/|G\s*B)', expand=False)
Laptops['RAM'] = Laptops['Laptop'].str.extract(r'(\d+)\s*(?:Gb/|G\s*B)', expand=False)

In [39]:
Laptops['RAM'].value_counts()

16     928
8      813
32     301
4       68
64      25
12      15
6        3
40       2
128      1
512      1
Name: RAM, dtype: int64

In [40]:
null_values = Laptops[Laptops['RAM'].isna()]
null_values['Laptop'].tolist()

['Portátil Alurin Flex Advance Intel Core I5-1155G7 14"',
 'Alurin AMD R5 5500U 15.6" 8 256 con SO',
 'Alurin Intel Core I7 12th 15.6" 16 500 con SO']

After that, we see there is 3 empty values in RAM. In order to simplify it, we fill it with the value '8'.

In [41]:
Laptops['RAM'] = Laptops['RAM'].fillna(8)

In [42]:
null_values = Laptops[Laptops['RAM'].isna()]
null_values['Laptop'].tolist()

[]

Now we got it. However, we have a value of RAM of 512. Seems weird, let's see it in detail:

In [43]:
filtered_df = Laptops.loc[Laptops['RAM'] == '512']
filtered_df

,Laptop,New Price,Old Price,Price,Old Price 2,Status,Final Price,Brand,CPU,Model,RAM
1136,"Apple MacBook Intel Core i5/512GB/12"" Gris Esp...",NaN,NaN,684.65,NaN,Reacondicionado,684.65,Apple,Intel Core i5,Macbook,512


Actually this Lap has a RAM of 8. We replace it:

In [44]:
Laptops['RAM'] = Laptops['RAM'].replace('512', '8')

In [45]:
Laptops['RAM'].value_counts()

16     928
8      814
32     301
4       68
64      25
12      15
8        3
6        3
40       2
128      1
Name: RAM, dtype: int64

Now it's perfect!

## Status

Now, we must simplify the Column Status. We just need two values. Refurbished, and new:

In [46]:
Laptops['Status'] = Laptops['Status'].replace('Reacondicionado', 'Refurbished')
Laptops['Status'] = Laptops['Status'].where(Laptops['Status'] == 'Refurbished', 'New')

Laptops[['Laptop','Status','Brand','Model','CPU','RAM','Final Price']]

,Laptop,Status,Brand,Model,CPU,RAM,Final Price
0,ASUS ExpertBook B1 B1502CBA-EJ0436X Intel Core...,New,Asus,ExpertBook,Intel Core i5,8,1009.00
1,Alurin Go Start Intel Celeron N4020/8GB/256GB ...,New,Alurin,Go,Intel Celeron,8,299.00
2,ASUS ExpertBook B1 B1502CBA-EJ0424X Intel Core...,New,Asus,ExpertBook,Intel Core i3,8,789.00
3,MSI Katana GF66 12UC-082XES Intel Core i7-1270...,New,MSI,Katana,Intel Core i7,16,1199.00
4,HP 15S-FQ5085NS Intel Core i5-1235U/16GB/512GB...,New,HP,15S,Intel Core i5,16,669.01
...,...,...,...,...,...,...,...
2159,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,Refurbished,Razer,Blade,Intel Core i7,16,2699.99
2160,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,Refurbished,Razer,Blade,Intel Core i7,16,2899.99
2161,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,Refurbished,Razer,Blade,Intel Core i7,32,3399.99
2162,Razer Book 13 Intel Evo Core i7-1165G7/16GB/1T...,Refurbished,Razer,Book,Intel Evo Core i7,16,1899.99


In [47]:
Laptops['Status'].value_counts()

New            1498
Refurbished     662
Name: Status, dtype: int64

In [48]:
null_values = Laptops[Laptops['Status'].isna()]
null_values['Laptop'].tolist()

[]

Perfect!

## Storage type

Now let's see how extract the Storage type. First of all, let' focus on HDD, SSD and eMMC:

In [49]:
Laptops['Storage type'] = Laptops['Laptop'].apply(lambda x: 'HDD' if 'HDD' in x else ('SSD' if 'SSD' in x else ('eMMC' if 'eMMC' in x else ('eMMC' if 'EMMC' in x else ''))))

In [50]:
Laptops['Storage type'].value_counts()

SSD     2062
eMMC      56
          42
Name: Storage type, dtype: int64

In [51]:
filtered_df = Laptops.loc[Laptops['Storage type'] == '']
filtered_df['Laptop'].tolist()

['ASUS Chromebook CX1400CNA-BV0210 Intel Celeron N3350/8GB/32GB/14"',
 'Portátil Alurin Flex Advance Intel Core I5-1155G7 14"',
 'ASUS ROG Strix G16 G614JZ-N3008 Intel Core i7-13650HX/32GB/1TB/RTX 4080/16"',
 'Prixton Flex Pro Intel Celeron N4020/4GB/64GB/11.6" Táctil',
 'Apple MacBook Pro Intel Core i5/8GB/256GB/13" Plateado',
 'Alurin AMD R5 5500U 15.6" 8 256 con SO',
 'Alurin Intel Core I7 12th 15.6" 16 500 con SO',
 'ASUS F515EA-BQ1625W Intel Core i3-1115G4/8GB/256GB/15.6"',
 'HP Chromebook x360 11 G3 Education Edition Intel Celeron N4020/4GB/32GB/11.6" Táctil',
 'HP Chromebook 11 G9 Intel Celeron N4500/4GB/32GB/11.6"',
 'Alurin Flex Advance Intel Core i5-1155G7/8GB/256GB/15.6" + Windows 11 Home',
 'PcCom Revolt 3050 Intel Core i7-13700H/16GB/500GB/RTX 3050/15.6" + Windows 11 Home',
 'PcCom Revolt 3050 Intel Core i7-13700H/16GB/500GB/RTX 3050/15.6"',
 'Apple MacBook Pro Intel Core i5/8GB/128GB/13" Plateado',
 'Microsoft Surface Pro 7 Intel Core i5-1035G4/8 GB/256 GB/12.3" Negra',
 

We see there is 42 values without storage type. After checking it, it's correct since we don't see it in Laptop column.

## Storage

For the storage column, we can extract the values just after the first "GB/" string. Let's try it:

In [52]:
Laptops['Storage'] = Laptops['Laptop'].apply(lambda x: re.search(r'GB\/(.*?)\/', x).group(1) if re.search(r'GB\/(.*?)\/', x) else '')

In [53]:
Laptops['Storage'].value_counts()

512GB SSD                      909
1TB SSD                        559
256GB SSD                      403
128GB SSD                       42
2TB SSD                         41
64GB eMMC                       32
500GB SSD                       32
256 GB SSD                      27
512 GB SSD                      19
128GB eMMC                      13
256GB                           11
                                10
128GB                            8
32GB eMMC                        8
512GB                            5
32GB                             4
500GB                            4
64GB                             3
512GBSSD                         3
1TB+1TB SSD                      3
128GB EMMC                       2
128 GB SSD                       2
1 TB SSD                         2
256 SSD                          2
1TB                              2
64GB+256GB SSD                   1
512                              1
256 GB                           1
512GB SSD+32GB Intel

In [54]:
filtered_df = Laptops.loc[Laptops['Storage'] == '']
filtered_df['Laptop'].tolist()

['Portátil Alurin Flex Advance Intel Core I5-1155G7 14"',
 'Alurin AMD R5 5500U 15.6" 8 256 con SO',
 'Alurin Intel Core I7 12th 15.6" 16 500 con SO',
 'Acer Extensa 15 EX215-54 Intel Core i5-1135G7/8GB/256GB SSD',
 'ASUS VivoBook F1500EA-EJ3066W Intel Core i7-1165G7/8GB RAM/512GB SSD/15.6"',
 'Apple MacBook Pro Intel Core i5/16Gb/512GB SSD/13.3" Gris Espacial',
 'Apple MacBook Intel Core i5/512GB/12" Gris Espacial',
 "Dell XPS 15 9520 Intel Core i7-12700H/16GB RAM/1TB SSD/RTX 3050 Ti/15.6'' (PT)",
 'Vant Edge 3 Intel Core i7-1255U/40GB RAM/1TB SSD/14"',
 'Apple MacBook Pro Touch Bar Intel Core i7/16Gb/1TB SSD/13.3" Gris Espacial']

Only 10 empty values, quite good! Let's work on this pending values:

In [55]:
Laptops['Storage'] = Laptops['Laptop'].apply(lambda x: re.search(r'GB\/(.*?)\/', x).group(1) if re.search(r'GB\/(.*?)\/', x) else (re.search(r'(\d+\w+)\sSSD', x).group(1) if re.search(r'(\d+\w+)\sSSD', x) else ''))

In [56]:
Laptops['Storage'].value_counts()

512GB SSD                      909
1TB SSD                        559
256GB SSD                      403
128GB SSD                       42
2TB SSD                         41
64GB eMMC                       32
500GB SSD                       32
256 GB SSD                      27
512 GB SSD                      19
128GB eMMC                      13
256GB                           12
128GB                            8
32GB eMMC                        8
512GB                            7
1TB                              5
                                 4
32GB                             4
500GB                            4
1TB+1TB SSD                      3
64GB                             3
512GBSSD                         3
128 GB SSD                       2
1 TB SSD                         2
128GB EMMC                       2
256 SSD                          2
512                              1
64GB+256GB SSD                   1
256 GB                           1
512GB SSD+32GB Intel

In [57]:
filtered_df = Laptops.loc[Laptops['Storage'] == '']
filtered_df['Laptop'].tolist()

['Portátil Alurin Flex Advance Intel Core I5-1155G7 14"',
 'Alurin AMD R5 5500U 15.6" 8 256 con SO',
 'Alurin Intel Core I7 12th 15.6" 16 500 con SO',
 'Apple MacBook Intel Core i5/512GB/12" Gris Espacial']

Good! Now only 4 pending values. Three of them we can set it manually:

In [58]:
Laptops.loc[Laptops['Laptop'] == 'Alurin AMD R5 5500U 15.6" 8 256 con SO', 'Storage'] = '256'
Laptops.loc[Laptops['Laptop'] == 'Alurin Intel Core I7 12th 15.6" 16 500 con SO', 'Storage'] = '500'
Laptops.loc[Laptops['Laptop'] == 'Apple MacBook Intel Core i5/512GB/12" Gris Espacial', 'Storage'] = '512'

In [59]:
filtered_df = Laptops.loc[Laptops['Storage'] == '']
filtered_df['Laptop'].tolist()

['Portátil Alurin Flex Advance Intel Core I5-1155G7 14"']

Nice! Now let's clean the column. We must have only integers, expressed as GB values:

In [60]:
Laptops['Storage'] = Laptops['Storage'].str.replace(' GB', '')
Laptops['Storage'] = Laptops['Storage'].str.replace('GB', '')
Laptops['Storage'] = Laptops['Storage'].str.replace(' TB', '')
Laptops['Storage'] = Laptops['Storage'].str.replace('TB', '')
Laptops['Storage'] = Laptops['Storage'].str.replace(' SSD', '')
Laptops['Storage'] = Laptops['Storage'].str.replace('SSD', '')
Laptops['Storage'] = Laptops['Storage'].str.replace(' eMMC', '')
Laptops['Storage'] = Laptops['Storage'].str.replace('eMMC', '')
Laptops['Storage'] = Laptops['Storage'].str.replace(' EMMC', '')
Laptops['Storage'] = Laptops['Storage'].str.replace('EMMC', '')
Laptops['Storage'] = Laptops['Storage'].str.replace(' Intel Optane', '')
Laptops['Storage'] = Laptops['Storage'].str.replace('2565', '256')

In [61]:
Laptops['Storage'].value_counts()

512       940
1         566
256       448
128        67
2          41
500        37
64         35
32         13
1+1         3
4           1
1+256       1
240         1
3           1
2+2         1
512+1       1
512+32      1
64+256      1
            1
1+512       1
Name: Storage, dtype: int64

In [62]:
Laptops['Storage'] = Laptops['Storage'].apply(lambda x: x.split('+')[-1].strip())

In [63]:
Laptops['Storage'].value_counts()

512    941
1      570
256    450
128     67
2       42
500     37
64      35
32      14
         1
4        1
3        1
240      1
Name: Storage, dtype: int64

Now we normalize the values in TB, to GB:

In [64]:
Laptops['Storage'] = Laptops['Storage'].apply(lambda x: int(x) * 1000 if x and int(x) < 10 else x)

In [65]:
Laptops['Storage'].value_counts()

512     941
1000    570
256     450
128      67
2000     42
500      37
64       35
32       14
          1
4000      1
3000      1
240       1
Name: Storage, dtype: int64

In [66]:
Laptops['Storage'] = pd.to_numeric(Laptops['Storage'], errors='coerce')

In [67]:
Laptops['Storage'] = Laptops['Storage'].fillna(0)
Laptops['Storage'] = Laptops['Storage'].astype(int)
Laptops['Storage'].value_counts()

512     941
1000    570
256     450
128      67
2000     42
500      37
64       35
32       14
0         1
4000      1
3000      1
240       1
Name: Storage, dtype: int64

Perfect!

## Screen

Let's work on the Screen column. The strategy is to extract the digits just before '"':

In [68]:
Laptops['Screen'] = Laptops['Laptop'].apply(lambda x: re.search(r'/(?P<screen>[^/]+)"', x).group('screen') if re.search(r'/(?P<screen>[^/]+)"', x) else re.search(r'(\d+(?:\.\d+))', x).group() if re.search(r'(\d+(?:\.\d+))', x) else re.search(r'(\d+)"', x).group(1) if re.search(r'(\d+)"', x) else re.search(r"(\d+)'", x).group(1) if re.search(r"(\d+)'", x) else '')

In [69]:
Laptops['Screen'].value_counts()

15.6                                            1008
14                                               392
16                                               173
17.3                                             161
13.3                                             131
16.1                                              47
17                                                33
13                                                27
15                                                21
13.4                                              19
13.5                                              19
11.6                                              16
14.2                                              14
12.3                                              13
13.6                                              11
14.1                                              11
16.2                                              10
15.3                                               8
10.5                                          

In [70]:
Laptops['Screen'] = Laptops['Screen'].str.replace(' Razer Soporte para Portátil hasta 15', '')
Laptops['Screen'] = Laptops['Screen'].str.replace('" ', '')
Laptops['Screen'] = Laptops['Screen'].str.replace('+', '')
Laptops['Screen'] = Laptops['Screen'].str.replace(',', '.')

C:\Users\juanm\AppData\Local\Temp\ipykernel_11560\1109816736.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Laptops['Screen'] = Laptops['Screen'].str.replace('+', '')


In [71]:
Laptops['Screen'].value_counts()

15.6     1009
14        392
16        173
17.3      161
13.3      131
16.1       48
17         33
13         27
15         21
13.4       19
13.5       19
11.6       16
14.2       14
12.3       13
13.6       11
14.1       11
16.2       10
15.3        8
10.5        7
14.4        6
12.4        6
15.4        5
            4
12          4
14.5        3
18          3
13.9        2
12.5        1
16.0        1
10.1        1
10.95       1
Name: Screen, dtype: int64

In [72]:
filtered_df = Laptops.loc[Laptops['Screen'] == '']
filtered_df['Laptop'].tolist()

['Acer Extensa 15 EX215-54 Intel Core i5-1135G7/8GB/256GB SSD',
 'HP ENVY x360 2-in-1 Laptop 15-ew0008np Intel Core i7-1255U/16GB/512GB SSD/RTX2050/Tátil (PT)',
 'Lenovo IdeaPad Gaming 3 15ACH6 AMD Ryzen 5 5600H/16GB/512GB SSD/RTX 3060',
 'Lenovo ThinkPad P15 Gen 2 Intel Core i7-11850H/16GB/512GB SSD/RTX A2000']

We have 4 rows that we don't have data about the Screen size. Let's convert the column to numeric:

In [73]:
Laptops['Screen'] = Laptops['Screen'].fillna(0)
Laptops['Screen'] = pd.to_numeric(Laptops['Screen'], errors='coerce')

In [74]:
Laptops['Screen'].value_counts()

15.60    1009
14.00     392
16.00     174
17.30     161
13.30     131
16.10      48
17.00      33
13.00      27
15.00      21
13.50      19
13.40      19
11.60      16
14.20      14
12.30      13
14.10      11
13.60      11
16.20      10
15.30       8
10.50       7
14.40       6
12.40       6
15.40       5
12.00       4
18.00       3
14.50       3
13.90       2
12.50       1
10.10       1
10.95       1
Name: Screen, dtype: int64

## GPU

Now let's create the column GPU. Since there is many possible values, the best approach is to use a dictionary gpu.xlsx:

In [75]:
path_gpu = os.path.join('master_data', 'gpu.xlsx')

gpu_df = pd.read_excel(path_gpu)
gpu_df

,Keyword,GPU
0,RTX 3060,RTX 3060
1,RTX 4070,RTX 4070
2,RTX 3070,RTX 3070
3,RTX 3050,RTX 3050
4,RTX 4060,RTX 4060
...,...,...
93,A730M,A 730M
94,RX555X,RX 555X
95,T2000,T 2000
96,610M,610 M


In [76]:
gpu_dict = gpu_df.set_index('Keyword')['GPU'].to_dict()

def assign_gpu(row):
    for keyword, gpu in gpu_dict.items():
        if keyword.lower() in row['Laptop'].lower():
            return gpu
    return None

Laptops['GPU'] = Laptops.apply(assign_gpu, axis=1)

In [77]:
Laptops['GPU'].value_counts()

RTX 3050              129
RTX 3060              122
RTX 3070               97
RTX 4060               62
RTX 3080               51
GTX 1650               50
RTX 4070               40
RTX 4050               33
RTX 2070               25
RTX 2060               20
RTX 4080               18
RTX 4090               17
GTX 1660               14
RTX 2050               11
Radeon Pro 5500M        9
RTX A2000               8
RTX 2080                7
MX 450                  7
RTX A3000               7
RTX A1000               6
T 1200                  6
MX 550                  5
MX 330                  5
T 500                   5
T 550                   4
GTX 1050                3
Radeon Pro RX 560X      3
RTX A5500               3
RX 6500M                3
RX 7600S                2
T 600                   2
Radeon Pro 5300M        2
610 M                   2
RX 6700M                1
P 500                   1
MX 130                  1
A 370M                  1
RX 6800S                1
T 2000      

It looks perfect! Not all the Laptops has a GPU, so the values seems to be nice.

## Touch

The last column is the column Touch. In this case, let's search for the words "Táctil", "Tactil", and similars. If so, the value would be Yes, if not, No:

In [78]:
Laptops['Touch'] = Laptops['Laptop'].apply(lambda x: 'Yes' if 'Táctil' in x else ('Yes' if 'Tactil' in x else ('Yes' if 'Tátil' in x else ('Yes' if 'Tatil' in x else 'No'))))

In [79]:
Laptops['Touch'].value_counts()

No     1933
Yes     227
Name: Touch, dtype: int64

We got it! It makes sense, that just a few laptops has a touch screen.

In [80]:
Laptops

,Laptop,New Price,Old Price,Price,Old Price 2,Status,Final Price,Brand,CPU,Model,RAM,Storage type,Storage,Screen,GPU,Touch
0,ASUS ExpertBook B1 B1502CBA-EJ0436X Intel Core...,699.00,1009.00,NaN,0.0,New,1009.00,Asus,Intel Core i5,ExpertBook,8,SSD,512,15.6,None,No
1,Alurin Go Start Intel Celeron N4020/8GB/256GB ...,229.00,299.00,NaN,0.0,New,299.00,Alurin,Intel Celeron,Go,8,SSD,256,15.6,None,No
2,ASUS ExpertBook B1 B1502CBA-EJ0424X Intel Core...,549.00,789.00,NaN,0.0,New,789.00,Asus,Intel Core i3,ExpertBook,8,SSD,256,15.6,None,No
3,MSI Katana GF66 12UC-082XES Intel Core i7-1270...,899.00,1199.00,NaN,0.0,New,1199.00,MSI,Intel Core i7,Katana,16,SSD,1000,15.6,RTX 3050,No
4,HP 15S-FQ5085NS Intel Core i5-1235U/16GB/512GB...,NaN,NaN,669.01,NaN,New,669.01,HP,Intel Core i5,15S,16,SSD,512,15.6,None,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,2151.39,2699.99,NaN,0.0,Refurbished,2699.99,Razer,Intel Core i7,Blade,16,SSD,1000,17.3,RTX 3060,No
2160,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,2016.39,2899.99,NaN,0.0,Refurbished,2899.99,Razer,Intel Core i7,Blade,16,SSD,1000,17.3,RTX 3070,No
2161,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,2069.69,3399.99,NaN,0.0,Refurbished,3399.99,Razer,Intel Core i7,Blade,32,SSD,1000,17.3,RTX 3080,No
2162,Razer Book 13 Intel Evo Core i7-1165G7/16GB/1T...,1515.19,1899.99,NaN,0.0,Refurbished,1899.99,Razer,Intel Evo Core i7,Book,16,SSD,1000,13.4,None,Yes


# Generating the csv file

The last part is generate the final file. We are going to generate a csv format file. But first of all, we need to reorganize the columns:

In [81]:
Laptops_csv = Laptops[['Laptop','Status','Brand','Model','CPU','RAM','Storage','Storage type','GPU','Screen','Touch','Final Price']]

In [82]:
Laptops_csv

,Laptop,Status,Brand,Model,CPU,RAM,Storage,Storage type,GPU,Screen,Touch,Final Price
0,ASUS ExpertBook B1 B1502CBA-EJ0436X Intel Core...,New,Asus,ExpertBook,Intel Core i5,8,512,SSD,None,15.6,No,1009.00
1,Alurin Go Start Intel Celeron N4020/8GB/256GB ...,New,Alurin,Go,Intel Celeron,8,256,SSD,None,15.6,No,299.00
2,ASUS ExpertBook B1 B1502CBA-EJ0424X Intel Core...,New,Asus,ExpertBook,Intel Core i3,8,256,SSD,None,15.6,No,789.00
3,MSI Katana GF66 12UC-082XES Intel Core i7-1270...,New,MSI,Katana,Intel Core i7,16,1000,SSD,RTX 3050,15.6,No,1199.00
4,HP 15S-FQ5085NS Intel Core i5-1235U/16GB/512GB...,New,HP,15S,Intel Core i5,16,512,SSD,None,15.6,No,669.01
...,...,...,...,...,...,...,...,...,...,...,...,...
2159,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,Refurbished,Razer,Blade,Intel Core i7,16,1000,SSD,RTX 3060,17.3,No,2699.99
2160,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,Refurbished,Razer,Blade,Intel Core i7,16,1000,SSD,RTX 3070,17.3,No,2899.99
2161,Razer Blade 17 FHD 360Hz Intel Core i7-11800H/...,Refurbished,Razer,Blade,Intel Core i7,32,1000,SSD,RTX 3080,17.3,No,3399.99
2162,Razer Book 13 Intel Evo Core i7-1165G7/16GB/1T...,Refurbished,Razer,Book,Intel Evo Core i7,16,1000,SSD,None,13.4,Yes,1899.99


In [83]:
Laptops_csv.to_csv('laptops.csv', index=False)